## Spam Detector

This project consists of two tasks:
1) to find spam-messages in the set of texts and remove them
2) to build the model which will be able to detect spam-messages

To solve these tasks I used pretrained Spacy model for Russian language and my own module in Python `nk_nlp1_5`  

This module contains two classes:  
`TextPreprocessing` - this class has several methods which helps to process texts using such operations like: regular expressions, deduplication, mapping, quoting and NLP-methods based on semantic similarity, finding part-of-speech and sentence dependences, named entity recognition and allows applying these methods to the collection of texts directly  

`Categorizator` - the class whose methods can identify dependencies between a sets of texts. It supports several approaches to the similarity calculation which can be specified by the special parameters. In addition to the similarity the methods allows you to calculate quoting - how often this word or expression occurs in the other set of texts. This can help you to concentrate your attention on the most significant objects.

See help(classname) for details.

In [1]:
import pandas as pd
import numpy as np
import json
import tqdm
from glob import glob

import spacy

# my module for text processing and categorization based on SpaCy
from nk_nlp1_5 import TextPreprocessing, Categorizator

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_row', 1000)
pd.set_option('display.max_column', 100)
pd.set_option('display.max_colwidth', None)

## Data loading and preprocessing

In [2]:
# func for the replacement of incorrect characters

def alphabet_replacer(string, to='ru'):
    ru = list('абдеикмнорстухАВЕКМНОРСТУХа')
    en = list('abdeukmnopctyxABEKMHOPCTYXα')
    if to == 'ru':
        for char in en:
            if char in string:
                string = string.replace(char, ru[en.index(char)])
    elif to == 'en':
        for char in ru:
            if char in string:
                string = string.replace(char, en[ru.index(char)])
    return string
    

In [3]:
# loading data from JSONL format

messages = []

for path in sorted(glob('../PT/chat_data/*.jsonl')):
    with open(path, 'r', encoding='utf8') as file:
        for line in file:
            messages.append(json.loads(line))
          

In [4]:
# extracting needed data from json

data_for_df = []

for message in messages:
    record = {
        'message_id': message['message'].get('message_id') if 'message' in message else None,
        'text': message['message'].get('text') if 'text' in message['message'] else None
    }
    
    data_for_df.append(record)

df = pd.DataFrame(data_for_df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3824 entries, 0 to 3823
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message_id  3824 non-null   int64 
 1   text        3268 non-null   object
dtypes: int64(1), object(1)
memory usage: 59.9+ KB


In [5]:
# NAN preprocessing

df = df[df.text.notna()]

In [6]:
# incorrect character replacing

df['text_repl'] = df['text'].apply(alphabet_replacer).str.lower()
df = df.reset_index()

In [10]:
df.loc[25:31, 'text':'text_repl']

,text,text_repl
25,Если в эту задачу больше ничего не летит - можно trace включить и будет видно что точно пришло и т д,если в эту задачу больше ничего не летит - можно тrасе включить и будет видно что точно пришло и т д
26,Задача завершится с ошибкой,задача завершится с ошибкой
27,"да, в эту задачу ничего не летит. Вы имеете в виду trace - утилита, которая позволяет проследить маршрут следования данных до удаленного адресата в сетях TCP/IP?","да, в эту задачу ничего не летит. вы имеете в виду тrасе - утилита, которая позволяет проследить маршрут следования данных до удаленного адресата в сетях тср/iр?"
28,"Набираю людей в команду для взаимовыгодного сотрудничества в CrУpТo\nСо старта получается от 2315$ в неделю\nПо времени - в день занимает до 2-х часов\nМожно Получать пассивный доход с любой точки мира!\nОт 18-ти лет‼️\n\nНапиши мне, если заинтересовала✉️","набираю людей в команду для взаимовыгодного сотрудничества в сrурто\nсо старта получается от 2315$ в неделю\nпо времени - в день занимает до 2-х часов\nможно получать пассивный доход с любой точки мира!\nот 18-ти лет‼️\n\nнапиши мне, если заинтересовала✉️"
29,"нет, можно в задаче сбора (logging_settings) настроить расширенный журнал и там будет все что прилетает и обрабатывается","нет, можно в задаче сбора (lоggiнg_sеттiнgs) настроить расширенный журнал и там будет все что прилетает и обрабатывается"
30,"Хорошо, поняла, сейчас попробуем, спасибо!","хорошо, поняла, сейчас попробуем, спасибо!"
31,"Добрый день!\nверсия 25 сертифицированная\nКак выпустить отчет по уязвимостям?\nРанее в 24й версии было проще - заходишь в активы, выбираешь нужные, и создать отчет по уязвимостям","добрый день!\nверсия 25 сертифицированная\nкак выпустить отчет по уязвимостям?\nранее в 24й версии было проще - заходишь в активы, выбираешь нужные, и создать отчет по уязвимостям"


## Spam detection on similarity

In [73]:
# loading the pretrained model

nlp = spacy.load('ru_core_news_lg')

Below we use my class `TextPreprocessing` for data processing. It contains several methods for data processing using NLP. Method `.word_extractor` is used to remain the specified number of the words which are closest to the set of words or phrases given as argument 'pattern'. We can also put some additional parameters into this method to filter words more finely. In the example below I use filtering by parts-of-speech.

In [74]:
tp = TextPreprocessing(nlp=nlp, text_col=df['text_repl'])

In [75]:
search_phrase = 'заработок, доход, прибыль, профит, трейдинг, перспектива, подработка, сотрудничество, выгода, биржа, удаленная, ищу людей, поиск партнеров, ломбард, деньги, купюры'

In [76]:
extr_results = tp.word_extractor(pattern=search_phrase, threshold=None, count_thres=10, dep=None, pos=['NOUN', 'VERB'], desc_sim=True, stat=False, full_df=False, aliquot=10)

In [77]:
df['expr_results'] = extr_results

In [78]:
df.loc[32:36, 'text_repl':'expr_results']

,text_repl,expr_results
32,"добрый день!\nподскажите пожалуйста, как в 25-й версии получить токен доступа, если уз авторизуется по lдар? в 26 тоже не совсем ясно из документации какие значения принимает параметр амr при получении токена.",получить получении доступа значения принимает день версии параметр подскажите уз
33,"желательно без пост запросов к :3334/иi/lоgiн/, а именно вариант с получением токена",вариант получением запросов пост
34,"коллеги, день добрый. никто не в курсе, разрабатывают нормализацию под линуксовый кsс? там же маriаdв, в дефолте никак не подружишь. по сислогу, насколько я понимаю, маловато инфы.",понимаю курсе день разрабатывают коллеги нормализацию
35,в 25- никак\nв актуальных версиях\nhттрs://hеlр.ртsесиriту.сом/рrоjестs/махратrоl10/26.2/rи-ru/hеlр/3678991755\nпараметр амr,параметр версиях
36,"ага, старый метод через эмуляцию ui и без использования сlент_sесrет тоже работает и его можно использовать . пример работы есть в мрsiемliб",работает работы метод использования использовать пример


Here we use another my class `Categorizator` which helps to categorize text data by the similarity. The method `cat_sim` calculates similarity between the specified phrase and text_data and sorts them by descending the similarity.

In [79]:
cat_kw = Categorizator(pattern_list=df['expr_results'], nlp=nlp, only_w_vector=False)

No cat given. Use param "cat_list".
Categories without vectors: Series([], dtype: object)
Starting NLP-processing for pattern_list


pattern_list processed



In [81]:
spam_results_kw = cat_kw.cat_sim(cat=search_phrase, sim_func='adv', metric='mean_top5')

Using preprocessed pattern_list
Starting quotes counting...
No text data for quoting! Uze param "quoting".


In [82]:
result_df = df.join(spam_results_kw.sort_index())

In [83]:
result_df.sort_values(search_phrase, ascending=False)[['text', 'expr_results', search_phrase]].head(5)

,text,expr_results,"заработок, доход, прибыль, профит, трейдинг, перспектива, подработка, сотрудничество, выгода, биржа, удаленная, ищу людей, поиск партнеров, ломбард, деньги, купюры"
1996,"Добрый вечер, ищу людей на удаленный заработок с 18 лет \nЗП от 150$ в день\nПишите + в лс",заработок ищу людей лет день пишите вечер,0.818371
1435,"Здравствуйте, ищу людей в тиму.\nСвободный график👌\nПриятный заработок от 200 $ в день\nЕсли интересует подробная информация "" + "" в лс",заработок интересует ищу информация людей график день здравствуйте,0.818371
490,Ecть cпocoб получить дoxoд\nНа прокрутах биржи Bybit и Bitget\nПрибыль к капиталу +2-3%\nОбучаем новичков с 0.\nРаботаем без сторонних сайтов\nЕсть фото/видео инфоматериалы по связкам.\nИнтересует? тогда пиши в Лс.,прибыль доход получить интересует капиталу биржи есть сайтов работаем,0.812563
846,Приветcтвyю! Ищeм людeй - готoвых нa взaимoвыгoднoм сoтрудничестве получать хoрoший дoп. дoxoд пo нашeмy фpилaнc прoeкту.\nПo вoпpoсaм - пишитe в личныe coобщeния.,доход получать ищем сотрудничестве людей проекту пишите сообщения вопросам приветствую,0.804261
2449,"Ищем партнеров, готoвых нa взaимoвыгoднoм сoтрудничестве получать хoрoший дoп. дoxoд пo нашeмy прoeкту. Непoлная занятость. \nПo вoпpoсaм - пишите в лс",доход партнеров получать ищем сотрудничестве занятость проекту пишите вопросам,0.804261


__Almost all the spam is located in the top of the table and it helps to label this data easily and prepare training dataset.__

In [ ]:
# saving the result into excel for the verification.

result_df.to_excel('spam_results2.xlsx')

We've obtained a list of the texts, in the top of that all spam-messages locate. So, we easily can label it and get train data.

## Estimating the labeling result

In [84]:
# loading the verified data

spam_results_df = pd.read_excel('spam_results_checked.xlsx', index_col='Unnamed: 0')

In [85]:
spam_results_df[['text', 'predict', 'target']].head()

,text,predict,target
1435,"Здравствуйте, ищу людей в тиму.\nСвободный график👌\nПриятный заработок от 200 $ в день\nЕсли интересует подробная информация "" + "" в лс",1,1
1996,"Добрый вечер, ищу людей на удаленный заработок с 18 лет \nЗП от 150$ в день\nПишите + в лс",1,1
490,Ecть cпocoб получить дoxoд\nНа прокрутах биржи Bybit и Bitget\nПрибыль к капиталу +2-3%\nОбучаем новичков с 0.\nРаботаем без сторонних сайтов\nЕсть фото/видео инфоматериалы по связкам.\nИнтересует? тогда пиши в Лс.,1,1
275,Дeнь дoбpый!\n\nИщeм пapтнepoв в сфepe кpиптoвaлюты \n\n- Yдaлённo!\n- Рaбoтa с ПК/Тeлeфoнa\n- Дoхoд oт 500$ в нeдeлю\n\nМы пpeдoстaвляeм:\n- Пepспeктивy \n- Обyчeниe бeсплaтнo.\n- Пoддepжкa 24/7.\n- Стaбильный дoхoд\n\nEсли зaинтepeсoвaлись - пишитe в ЛС.,1,1
459,Приветcтвyю! Ищeм людeй - готoвых нa взaимoвыгoднoм сoтрудничестве получать хoрoший дoп. дoxoд пo нашeмy фpилaнc прoeкту.\nПo вoпpoсaм - пишитe в личныe coобщeния.,1,1


In [86]:
spam_pred = spam_results_df['predict']
spam_target = spam_results_df['target']

In [87]:
spam_target.value_counts()

target
0    3150
1     118
Name: count, dtype: int64

In [88]:
acc = accuracy_score(spam_target, spam_pred)
acc

0.9877600979192166

In [89]:
f1 = f1_score(spam_target, spam_pred)
f1

0.8095238095238095

As we can see from the metrics this methods of labeling shows a good result.

## Training model

We use the special method `get_train_data` of the class `TextPreprocessing` to prepare train data. For the model training we take pretrained Spacy model for Russian language and train it on a new data.

In [90]:
spam_target = pd.DataFrame({'SPAM': spam_target})
spam_target.head()

,SPAM
1435,1
1996,1
490,1
275,1
459,1


In [31]:
%%time
train = tp.get_train_data(label_data=spam_target.to_dict('records'), pattern_list=None, to_disk='./train_data/spam/',
                          split=0.2, label=None, text_col=spam_results_df['text_repl'], stratify=spam_target.squeeze('columns'))

Using label_data (list or Series with special dict


Splitting data: TRAIN - 80.0%,  TEST - 20.0%
Training data locates:
./train_data/spam/train.spacy
./train_data/spam/dev.spacy
CPU times: total: 27.6 s
Wall time: 27.5 s


In [32]:
%%time
# starting model training

spacy.cli.train.train("./config/config.cfg", "./TRAINED_MODEL/", overrides={"paths.train": "./train_data/spam/train.spacy", "paths.dev": "./train_data/spam/dev.spacy"})

✔ Created output directory: TRAINED_MODEL
ℹ Saving to output directory: TRAINED_MODEL
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler',
'lemmatizer', 'ner', 'textcat_multilabel']
ℹ Frozen components: ['tok2vec', 'morphologizer', 'parser', 'senter',
'attribute_ruler', 'lemmatizer', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_P  SENTS_R  SENTS_F  LEMMA_ACC  ENTS_F  ENTS_P  ENTS_R  CATS_SCORE  SCORE 
---  ------  -------------  -------  ---------  -------  -------  -------  -------  -------  ---------  ------  ------  ------  ----------  ------
  0       0           0.15    98.61      98.61    98.33    98.33   100.00   100.00   100.00      98.61  100.00  100.00  100.00       60.38    0.93
  3    1000          20.47   

## Model testing

In [91]:
# loading trained model

spam_model = spacy.load('./TRAINED_MODEL/spam_detector')

### Testing on the source data

The method `extract_cats` allows to categorize a text if it is spam or not using the trained model.

In [92]:
tp_spam = TextPreprocessing(text_col=df['text_repl'], nlp=spam_model)

In [93]:
spam_res = tp_spam.extract_cats(df=True)

In [94]:
spam_res.sort_values('SPAM', ascending=False).head()

,text_col,SPAM
1418,приветствую! ищем людей - готовых на взаимовыгодном сотрудничестве получать хороший доп. доход по нашему фриланс проекту.\nпо вопросам - пишите в личные сообщения.,1.0
3187,"набираю людей в команду для совместного заработка \nдоход в среднем 800$ в неделю, тратя до двух часов в день\nобучаем всех бесплатно\nинтересно попробовать? отправьте мне +",1.0
936,"хочешь изменить свою финансовую жизнь с минимумом усилий? узнай, как зарабатывать с пк, не выходя из дома! напиши + в лс и открой для себя мир возможностей и высоких доходов. не упусти шанс на финансовую независимость!",1.0
935,"приветствую! ищем людей - готовых на взаимовыгодном сотрудничестве получать хороший доп. доход по нашему фриланс проекту (сфера криптовалют, не занимаемся торговлей, арбитражем и т.д)\nпоможем разобраться на практике если нет опыта. \nот 20 лет. неполная занятость.\nпо вопросам - пишите ему в л.с: @кliм_sаzоni",1.0
934,"приветствую! ищем людей - готовых на взаимовыгодном сотрудничестве получать хороший доп. доход по нашему фриланс проекту (сфера криптовалют, не занимаемся торговлей, арбитражем и т.д)\nпоможем разобраться на практике если нет опыта. \nот 20 лет. неполная занятость.\nпо вопросам - пишите ему в л.с: @кliм_sаzоni",1.0


In [95]:
spam_target = spam_results_df.sort_index()['target']
spam_pred = (spam_res['SPAM'] > 0.6).astype('int')

In [96]:
acc = accuracy_score(spam_target, spam_pred)
acc

0.9954100367197063

In [97]:
f1 = f1_score(spam_target, spam_pred)
f1

0.9382716049382717

We can see that metrics are good and this result was fully expected

### Testing on new data

As a test dataset we are using a set of the generated texts which are very similar to the source texts, but the model has never seen them before.

In [98]:
new_data = pd.read_excel('generated_spam_test.xlsx')

In [99]:
new_data.head()

,Unnamed: 0,message_text
0,515,"Привет всем! У меня есть курсы по трейдингу, которыми я могу поделиться абсолютно бесплатно. Если кому-то нужно, дайте знать"
1,570,"Внимание! Нужен 1 сотрудник для работы на дому, оплата достойная. Подробности вышлю в личные сообщения."
2,561,Заходите к нам за заработком! Отправьте сообщение и узнайте как!
3,162,Всем привет! У нас есть способ поднять кэш. Мы предоставляем это за процент от суммы. Работа проводится дистанционно. Заинтересованные могут связаться с нами👈🏼
4,616,"Найдите интимные фото девушки, используя код cdy382 в Телеграме."


In [114]:
# replace incorrect characters if they are

new_data['text_repl'] = new_data['message_text'].apply(alphabet_replacer).str.lower()

In [101]:
# spam recognition

tp_spam = TextPreprocessing(text_col=new_data['text_repl'], nlp=spam_model)
spam_res = tp_spam.extract_cats(df=True)

In [38]:
spam_res

,text_col,SPAM
0,"привет всем! у меня есть курсы по трейдингу, которыми я могу поделиться абсолютно бесплатно. если кому-то нужно, дайте знать",0.998
1,"внимание! нужен 1 сотрудник для работы на дому, оплата достойная. подробности вышлю в личные сообщения.",1.000
2,заходите к нам за заработком! отправьте сообщение и узнайте как!,0.938
3,всем привет! у нас есть способ поднять кэш. мы предоставляем это за процент от суммы. работа проводится дистанционно. заинтересованные могут связаться с нами👈🏼,0.000
4,"найдите интимные фото девушки, используя код сду382 в телеграме.",0.001
5,грандиозные новости - приглашаем на удаленную работу в теsтнет! бесплатное обучение и первый доход уже через 30 минут. доход составит от 1035$/100000 руб в неделю! у нас полная прозрачность! 😋еще доступно 8 мест! набор идет до 25.2.2024. буду рад видеть вас в команде!🥰,0.932
6,"приветствую! ищу личностей, которые хотят стать частью нашей команды и работать с криптовалютами. в вашем доступе будет постоянный доход от 1575$ в неделю, при временных затратах всего 2 часа в день! заинтересованы и вам нет 18 лет? напишите мне!",0.997
7,"приём на работу! нужны три человека, работа через телефон или пк. вознаграждение от 500$. пишите ""+"" для отклика.",0.998
8,наш проект открывает возможности для получения доп. дохода на условиях взаимовыгодного сотрудничества. ищем активных партнеров! вопросы можете задать в личных сообщениях.,0.000
9,"приветствую! 👋 требуются партнеры в команду. от вас требуется интернет и смартфон. возможность заработать 135-185 $ в день. все действия законны! если вам интересно, пишите в лс.",0.998


In [39]:
acc = accuracy_score(np.ones(len(spam_res)), (spam_res['SPAM'] > 0.6).astype('int'))
acc

0.6625

Since we used only spam texts as a test dataset `f1`-metric doesn't make sense and we calculates only `accuracy`.  
Unfortunately the result isn't good and this reports us about a low generalization level of this model in the task of spam detection. Obviously that the reason for this problem is connected with small quantity of spam examples

## Training the model on the enriched dataset

In [103]:
800 / 3880

0.20618556701030927

I have used ChatGPT, YandexGPT to generate new spam messages similar to source messages.  
For this purpose I write the special script to connect with the api.  
Before this we had only 120 spam examples (3.8% of the whole dataset), but after enriching we have obtained about 800 spam examples (20.6%)

In [104]:
enr_df = pd.read_excel('enriched_dataset.xlsx')

In [105]:
enr_df.head()

,message_text,target
0,"Ваш шанс на заработок ожидает вас! Напишите нам прямо сейчас, чтобы узнать, как приступить!",1
1,"Ищу кандидатов на удаленную работу, доход от 230$ в день. Заинтересовались? Пишите мне в лс для получения детальной информации😊",1
2,"Здравствуйте! Предлагается удаленная подработка, где вы можете заработать от 90$ в день и больше. Опыт не требуется. Если интересно, сообщите об этом в лс с помощью +",1
3,Есть предложение для потенциальных партнеров в новый проект. Ожидаемый доход составляет примерно 950$ в неделю. Пишите в лс для более подробной информации.,1
4,"Привет всем! У меня есть материалы курса по трейдингу, который я прошёл год назад. Если вам будет интересно, могу отправить вам. Жду только ваше спасибо взамен.",1


In [107]:
enr_df['text_repl'] = enr_df['message_text'].apply(alphabet_replacer).str.lower()

In [108]:
spam_target = enr_df['target']

In [109]:
spam_tp2 = TextPreprocessing(text_col=enr_df['text_repl'], nlp=spam_model)

In [54]:
%%time

# obtaining new training data

train = spam_tp2.get_train_data(label_data=enr_df[['target']].to_dict(orient='records'), pattern_list=None, to_disk='./train_data/spam2/',
                          split=0.2, label=None, text_col=None, stratify=spam_target)

Using label_data (list or Series with special dict


Splitting data: TRAIN - 80.0%,  TEST - 20.0%
Training data locates:
./train_data/spam2/train.spacy
./train_data/spam2/dev.spacy
CPU times: total: 40.1 s
Wall time: 40.1 s


In [55]:
%%time

# training a model

spacy.cli.train.train("./config/config.cfg", "./TRAINED_MODEL/", overrides={"paths.train": "./train_data/spam2/train.spacy", "paths.dev": "./train_data/spam2/dev.spacy"})

ℹ Saving to output directory: TRAINED_MODEL
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler',
'lemmatizer', 'ner', 'textcat_multilabel']
ℹ Frozen components: ['tok2vec', 'morphologizer', 'parser', 'senter',
'attribute_ruler', 'lemmatizer', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_P  SENTS_R  SENTS_F  LEMMA_ACC  ENTS_F  ENTS_P  ENTS_R  CATS_SCORE  SCORE 
---  ------  -------------  -------  ---------  -------  -------  -------  -------  -------  ---------  ------  ------  ------  ----------  ------
  0       0           0.06    98.91      98.91    98.69    98.69   100.00   100.00   100.00      98.91  100.00  100.00  100.00       51.38    0.91
  2    1000          24.76    98.91      98.91    98.69    98.69   100.

## Testing the trained model

In [110]:
spam_model = spacy.load('./TRAINED_MODEL/spam_detector2')

### Testing on new data

__Taking those test dataset that we used for the previous model__

In [116]:
tp_spam = TextPreprocessing(text_col=new_data['text_repl'], nlp=spam_model)

In [117]:
spam_res = tp_spam.extract_cats(df=True)

In [118]:
spam_res

,text_col,target
0,"привет всем! у меня есть курсы по трейдингу, которыми я могу поделиться абсолютно бесплатно. если кому-то нужно, дайте знать",1.000
1,"внимание! нужен 1 сотрудник для работы на дому, оплата достойная. подробности вышлю в личные сообщения.",1.000
2,заходите к нам за заработком! отправьте сообщение и узнайте как!,1.000
3,всем привет! у нас есть способ поднять кэш. мы предоставляем это за процент от суммы. работа проводится дистанционно. заинтересованные могут связаться с нами👈🏼,1.000
4,"найдите интимные фото девушки, используя код сду382 в телеграме.",1.000
5,грандиозные новости - приглашаем на удаленную работу в теsтнет! бесплатное обучение и первый доход уже через 30 минут. доход составит от 1035$/100000 руб в неделю! у нас полная прозрачность! 😋еще доступно 8 мест! набор идет до 25.2.2024. буду рад видеть вас в команде!🥰,0.994
6,"приветствую! ищу личностей, которые хотят стать частью нашей команды и работать с криптовалютами. в вашем доступе будет постоянный доход от 1575$ в неделю, при временных затратах всего 2 часа в день! заинтересованы и вам нет 18 лет? напишите мне!",1.000
7,"приём на работу! нужны три человека, работа через телефон или пк. вознаграждение от 500$. пишите ""+"" для отклика.",1.000
8,наш проект открывает возможности для получения доп. дохода на условиях взаимовыгодного сотрудничества. ищем активных партнеров! вопросы можете задать в личных сообщениях.,1.000
9,"приветствую! 👋 требуются партнеры в команду. от вас требуется интернет и смартфон. возможность заработать 135-185 $ в день. все действия законны! если вам интересно, пишите в лс.",1.000


In [119]:
acc = accuracy_score(np.ones(len(spam_res)), (spam_res['target'] > 0.6).astype('int'))
acc

0.99375

The metric is excellent and the generalization of this model is significantly better than the generalization of the previous one.  
Spam filter for this chat is ready.